purpose: run MAGMA on human control traits (taken from the ext1.0 paper's traits that were least and most significantly associated with externalizing)

# set-up

In [1]:
import pandas as pd
import os
import glob
import statsmodels.stats.multitest

In [2]:
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/gwas_ctrl_hm')

# import datasets

In [3]:
meta_data=pd.read_csv('ext1_st8_corr_sources_downloaded.csv')

In [4]:
meta_data

,Trait 1,Trait 2,N trait 2,rG,abs(rG),SE,Z,P value,Source,Domain,Subdomain,Phenotype,N,Prepared?,paper,Download,label,ctrl_type
0,Externalizing,Townsend deprivation index*,"112,151",0.707,0.707,0.047,15.167,5.820000e-52,CCACE,Socioeconomic,Economic & Environmental,Neighborhood deprivation (Townsend Index),112151,YES,NaN,http://www.psy.ed.ac.uk/ccace/downloads/Hill_C...,townsend,positive
1,Externalizing,Antisocial behavior,"16,400",0.647,0.647,0.166,3.906,9.400000e-05,CTG Lab,Psychopathology,Psychiatric Symptoms,Antisocial behavior,16400,YES,NaN,http://broadabc.ctglab.nl/documents/p12/BroadA...,antisoc,positive
2,Externalizing,Age of smoking initiation*,"262,990",-0.646,0.646,0.026,-24.708,8.760000e-135,GSCAN,Psychopathology,Substance Use,Age of smoking initiation,262990,YES,NaN,https://conservancy.umn.edu/handle/11299/201564,age_smkinit,positive
3,Externalizing,Friendships satisfaction*,"361,194",-0.001,0.001,0.032,-0.039,9.680000e-01,GWAS Atlas,Socioeconomic,Social,Friendships satisfaction,127735,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.4570.0...,friend_sat,negative
4,Externalizing,Infant birth weight*,"361,194",-0.012,0.012,0.020,-0.609,5.420000e-01,GWAS Atlas,Health & Medicine,Anthropomorphic,Infant birth weight,219088,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.20022....,infant_bw,negative
5,Externalizing,Heart rate*,"361,194",-0.026,0.026,0.018,-1.426,1.540000e-01,GWAS Atlas,Health & Medicine,Cardiometabolic,Heart rate,361411,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.102.0....,hr,negative
6,Externalizing,Age at first facial hair (male)*,"167,020",0.033,0.033,0.024,1.420,1.560000e-01,GWAS Atlas,Demography,Life History & Reproduction,Relative age of first facial hair (male),171805,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.2375.0...,facial_hair,negative
7,Externalizing,Maternal smoking around birth*,"361,194",0.717,0.717,0.021,34.403,2.240000e-259,GWAS Atlas,Socioeconomic,Economic & Environmental,Maternal smoking around birth,331862,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.1787.0...,maternal_smok,positive
8,Externalizing,Age at menarche*,"194,174",-0.010,0.010,0.021,-0.461,6.450000e-01,ReproGen,Demography,Life History & Reproduction,Age at menarche,252514,YES,NaN,http://www.reprogen.org/Menarche_1KG_NatGen201...,age_menarche,negative
9,Externalizing,LDL,"173,081",0.025,0.025,0.032,0.768,4.430000e-01,GLGC,Health & Medicine,Cardiometabolic,LDL Cholesterol,188577,YES,NaN,http://csg.sph.umich.edu/abecasis/public/lipid...,LDL,negative


In [ ]:
file_dict={'facial_hair':pd.read_csv('age_facial_hair_male.txt.gz',compression='gzip',sep='\t'),
    'age_smkinit':pd.read_csv('age_smkinit.txt.gz',compression='gzip',sep='\t'),
    'antisoc':pd.read_csv('antisoc_behav.tbl',sep='\t',low_memory=False),
    'friend_sat':pd.read_csv('friendship_satisfaction.txt.gz',compression='gzip',sep='\t'),
    'hr':pd.read_csv('heart_rate.txt',sep='\t'),
    'infant_bw':pd.read_csv('infant_birth_weight.txt.gz',compression='gzip',sep='\t'),
    'LDL':pd.read_csv('jointGwasMc_LDL.txt.gz',compression='gzip',sep='\t'),
    'maternal_smok':pd.read_csv('maternal_smok.txt',sep='\t'),
    'townsend':pd.read_csv('townsend_Hill_CB_2016/Hill2016_UKB_Income_summary_results_21112016.txt',sep=' '),
    'age_menarche':pd.read_csv('age_menarche/Menarche_1KG_NatGen2017_WebsiteUpload.txt',delim_whitespace=True),
    'neurot':pd.read_csv('neuroticism/GPC-2.NEUROTICISM.full.txt',sep='\t',header=None),
    'addict-rf':pd.read_csv('hatoum_addiction_euro_2023.txt.gz',sep='\t'),
    'cud':pd.read_csv('CUD_EUR_full_public_11.14.2020.gz',compression='gzip',delim_whitespace=True),
    'adhd':pd.read_csv('adhd.txt',sep='\t'),
    'alc_dep':pd.read_csv('pgc_alcdep.discovery.aug2018_release.txt.gz',delim_whitespace=True),
    'dpw':pd.read_csv('DRINKS_PER_WEEK_GWAS.txt',sep='\t'),
    'risk':pd.read_csv('RISK_GWAS_MA_UKB+replication.txt',sep='\t'),
    'auto_speed':pd.read_csv('AUTOMOBILE_SPEEDING_PROPENSITY_GWAS.txt',sep='\t'),
    'nsex':pd.read_csv('NUMBER_SEXUAL_PARTNERS_GWAS.txt',sep='\t'),
    'bmi':pd.read_csv('bmi.giant-ukbb.meta-analysis.combined.23May2018.txt.gz',compression='gzip',delim_whitespace=True),
    'height':pd.read_csv('Meta-analysis_Wood_et_al+UKBiobank_2018.txt.gz',compression='gzip',sep='\t')
}

# format files

## addiction risk factor

In [ ]:
file_dict['addict-rf']=file_dict['addict-rf'].dropna()

In [46]:
file_dict['addict-rf']['Chr']=file_dict['addict-rf']['Chr'].apply(lambda s: int(s))
file_dict['addict-rf']['BP']=file_dict['addict-rf']['BP'].apply(lambda s: int(s))

## antisocial

In [6]:
file_dict['antisoc']['CHR']=file_dict['antisoc']['MarkerName'].apply(lambda s: s.split(':')[0])
file_dict['antisoc']['BP']=file_dict['antisoc']['MarkerName'].apply(lambda s: s.split(':')[1])

## neuroticism

In [7]:
file_dict['neurot'].columns=['SNPID','CHR','BP','A1','A2','BETA','SE','PVALUE','NCOH','MAF']
#from the neuroticism read me

## age_menarche

In [8]:
file_dict['age_menarche']['CHR']=file_dict['age_menarche']['Markername'].apply(lambda s: s.split(':')[0][3:])
file_dict['age_menarche']['BP']=file_dict['age_menarche']['Markername'].apply(lambda s: s.split(':')[1])

## LDL

In [9]:
file_dict['LDL']['CHR']=file_dict['LDL']['SNP_hg19'].apply(lambda s: s.split(':')[0][3:])
file_dict['LDL']['BP']=file_dict['LDL']['SNP_hg19'].apply(lambda s: s.split(':')[1])

## alcohol dependence

In [35]:
file_dict['alc_dep']['SNPID']=file_dict['alc_dep']['SNP'].apply(lambda s: s.split(':')[0])

## bmi

In [77]:
file_dict['bmi']['SNP']=file_dict['bmi']['SNP'].apply(lambda s: s.split(':')[0])

In [81]:
file_dict['bmi']=file_dict['bmi'].dropna()

In [93]:
file_dict['bmi']['CHR']=file_dict['bmi']['CHR'].astype(int)

/tmp/ipykernel_404949/2975081436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_dict['bmi']['CHR']=file_dict['bmi']['CHR'].astype(int)


In [95]:
file_dict['bmi']['POS']=file_dict['bmi']['POS'].astype(int)

/tmp/ipykernel_404949/321537160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_dict['bmi']['POS']=file_dict['bmi']['POS'].astype(int)


## risk

In [123]:
file_dict['risk']=file_dict['risk'].dropna(subset=['MarkerName','CHR','POS','Pval'])

## dpw

In [158]:
len(file_dict['dpw'].dropna(subset=['MarkerName','CHR','POS','Pval']))/len(file_dict['dpw'])

0.9915649552893737

In [159]:
file_dict['dpw']=file_dict['dpw'].dropna(subset=['MarkerName','CHR','POS','Pval'])

## automotive speeding

In [153]:
len(file_dict['auto_speed'].dropna(subset=['MarkerName','CHR','POS','Pval']))/len(file_dict['auto_speed'])

0.9915660851921623

In [154]:
file_dict['auto_speed']=file_dict['auto_speed'].dropna(subset=['MarkerName','CHR','POS','Pval'])

## nsexual partners

In [153]:
len(file_dict['nsex'].dropna(subset=['MarkerName','CHR','POS','Pval']))/len(file_dict['nsex'])

0.9915660851921623

In [161]:
file_dict['nsex']=file_dict['nsex'].dropna(subset=['MarkerName','CHR','POS','Pval'])

# export magma input files

SNP  CHR        POS
0   rs1000000   12  126890980
1  rs10000003    4   57561647
2  rs10000011    4  138223055
3  rs10000013    4   37225069
4  rs10000015    4   84143987
pos file written to magma/addict-rf_pos.tsv
1025550
         N         SNP         P
0  1025550   rs1000000  0.701658
1  1025550  rs10000003  0.956505
2  1025550  rs10000011  0.693489
3  1025550  rs10000013  0.082888
4  1025550  rs10000015  0.766094

In [28]:
for f in file_dict.keys():
    print(f)
    print(file_dict[f].columns)

cud
Index(['CHR', 'SNP', 'BP', 'A1', 'A2', 'Z', 'P', 'N', 'N_CAS', 'N_CON'], dtype='object')
adhd
Index(['CHR', 'SNP', 'BP', 'A1', 'A2', 'FRQ_A_19099', 'FRQ_U_34194', 'INFO',
       'OR', 'SE', 'P', 'ngt', 'Direction', 'HetISqt', 'HetDf', 'HetPVa',
       'Nca', 'Nco', 'Neff'],
      dtype='object')
alc_dep
Index(['CHR', 'SNP', 'BP', 'A1', 'A2', 'Z', 'P', 'Weight'], dtype='object')
dpw
Index(['MarkerName', 'CHR', 'POS', 'A1', 'A2', 'EAF_A1', 'Beta', 'SE', 'Pval'], dtype='object')
risk
Index(['MarkerName', 'CHR', 'POS', 'A1', 'A2', 'EAF_A1', 'Beta', 'SE', 'Pval'], dtype='object')
auto_speed
Index(['MarkerName', 'CHR', 'POS', 'A1', 'A2', 'EAF_A1', 'Beta', 'SE', 'Pval'], dtype='object')
nsex
Index(['MarkerName', 'CHR', 'POS', 'A1', 'A2', 'EAF_A1', 'Beta', 'SE', 'Pval'], dtype='object')
bmi
Index(['CHR POS SNP Tested_Allele Other_Allele Freq_Tested_Allele BETA SE P N INFO'], dtype='object')
height
Index(['CHR', 'POS', 'SNP', 'Tested_Allele', 'Other_Allele',
       'Freq_Tested_Allele_in_HR

In [71]:
colname_dict={
    'facial_hair':['SNP','CHR','BP','P','A1','A2'],
    'age_smkinit':['RSID','CHROM','POS','PVALUE'],
    'antisoc':['MarkerName','CHR','BP','P-value','Allele1','Allele2'],
    'friend_sat':['SNP','CHR','BP','P','A1','A2'], 
    'hr':['SNP','CHR','BP','P','A1','A2'],
    'infant_bw':['SNP','CHR','BP','P','A1','A2'],
    'LDL':['rsid','CHR','BP','P-value'], 
    'maternal_smok':['SNP','CHR','BP','P','A1','A2'],
    'townsend':['Markername','Chromosome','Position','P-value'],
    'age_menarche':['Markername','CHR','BP','Pvalue','Allele1','Allele2'], 
    'neurot':['SNPID','CHR','BP','PVALUE'],
    'addict-rf':['SNP','Chr','BP','P','A1','A2'],
    'cud':['SNP','CHR','BP','P','A1','A2'],
    'adhd':['SNP','CHR','BP','P','A1','A2'],
    'alc_dep':['SNPID','CHR','BP','P','A1','A2'],
    'dpw':['MarkerName','CHR','POS','Pval','A1','A2'],
    'risk':['MarkerName','CHR','POS','Pval','A1','A2'],
    'auto_speed':['MarkerName','CHR','POS','Pval','A1','A2'],
    'nsex':['MarkerName','CHR','POS','Pval','A1','A2'],
    'bmi':['SNP','CHR','POS','P','Tested_Allele','Other_Allele'],
    'height':['SNP','CHR','POS','P','Tested_Allele','Other_Allele']
}

In [125]:
def ex_magma(snp,colname,N,prefix,dir,use_RSID=False):
    #purpose- format pvalue and pos files- specifics for mlma
    if use_RSID==False:
        pos=snp[[colname[0],colname[1],colname[2]]]
    else:
        pos=snp[['RSID',colname[1],colname[2]]]
    pos.columns=['SNP','CHR','POS']
    pos.to_csv(dir+prefix+'_pos.tsv',sep='\t',index=False)
    print(pos.head())
    print('pos file written to '+dir+prefix+'_pos.tsv')
    if use_RSID==False:
        pval=snp[[colname[0],colname[3]]]
    else:
        pval=snp[['RSID',colname[3]]]
    pval.columns=['SNP','P']
    #pval['N']=N
    print(N)
    pval.insert(0, 'N', N)
    print(pval.head())

    pval.to_csv(dir+prefix+'_pval.tsv',sep='\t',index=False)
    print('pval file written to '+dir+prefix+'_pval.tsv')
    return pos,pval

In [50]:
file_dict.keys()

dict_keys(['facial_hair', 'age_smkinit', 'antisoc', 'friend_sat', 'hr', 'infant_bw', 'LDL', 'maternal_smok', 'townsend', 'age_menarche', 'neurot', 'addict-rf'])

In [74]:
for t in file_dict.keys():
    print(t)
    pos,pval=ex_magma(file_dict[t],colname_dict[t],int(meta_data[meta_data['label']==t]['N']),t,'magma/')

cud
          SNP  CHR        POS
0  rs12562373    1  166367755
1   rs1023487    1  201297667
2  rs61226122    1  248287440
3  rs60128092    1   43612017
4   rs6664873    1   18039182
pos file written to magma/cud_pos.tsv
374287
        N         SNP        P
0  374287  rs12562373  0.03552
1  374287   rs1023487  0.27700
2  374287  rs61226122  0.18810
3  374287  rs60128092  0.89000
4  374287   rs6664873  0.13570
pval file written to magma/cud_pval.tsv
adhd
          SNP  CHR        POS
0  rs62513865    8  101592213
1  rs79643588    8  106973048
2  rs17396518    8  108690829
3    rs983166    8  108681675
4  rs28842593    8  103044620
pos file written to magma/adhd_pos.tsv
55374
       N         SNP        P
0  55374  rs62513865  0.02651
1  55374  rs79643588  0.69230
2  55374  rs17396518  0.58490
3  55374    rs983166  0.93650
4  55374  rs28842593  0.78780
pval file written to magma/adhd_pval.tsv
alc_dep
           SNP  CHR        POS
0  rs370839278    1  194108898
1  rs200328701    1   57

In [162]:
t='nsex'
pos,pval=ex_magma(file_dict[t],colname_dict[t],int(meta_data[meta_data['label']==t]['N']),t,'magma/')

           SNP  CHR     POS
0  rs114608975    1   86028
1   rs12238997    1  693731
2    rs4565649    1  713092
3  rs149887893    1  714596
4  rs141090730    1  715205
pos file written to magma/nsex_pos.tsv
370711
        N          SNP         P
0  370711  rs114608975  0.254300
1  370711   rs12238997  0.910800
2  370711    rs4565649  0.002521
3  370711  rs149887893  0.184000
4  370711  rs141090730  0.003381
pval file written to magma/nsex_pval.tsv


In [51]:
t='addict-rf'
pos,pval=ex_magma(file_dict[t],colname_dict[t],int(meta_data[meta_data['label']==t]['N']),t,'magma/')

          SNP  CHR        POS
0   rs1000000   12  126890980
1  rs10000003    4   57561647
2  rs10000011    4  138223055
3  rs10000013    4   37225069
4  rs10000015    4   84143987
pos file written to magma/addict-rf_pos.tsv
1025550
         N         SNP         P
0  1025550   rs1000000  0.701658
1  1025550  rs10000003  0.956505
2  1025550  rs10000011  0.693489
3  1025550  rs10000013  0.082888
4  1025550  rs10000015  0.766094
pval file written to magma/addict-rf_pval.tsv


# check output files for failed jobs

In [5]:
print(glob.glob("magma/*.annot"))

['magma/infant_bw.genes.annot', 'magma/hr.genes.annot', 'magma/alc_dep.genes.annot', 'magma/antisoc.genes.annot', 'magma/facial_hair.genes.annot', 'magma/addict-rf.genes.annot', 'magma/age_menarche.genes.annot', 'magma/neurot.genes.annot', 'magma/adhd.genes.annot', 'magma/bmi.genes.annot', 'magma/townsend.genes.annot', 'magma/nsex.genes.annot', 'magma/friend_sat.genes.annot', 'magma/height.genes.annot', 'magma/auto_speed.genes.annot', 'magma/cud.genes.annot', 'magma/dpw.genes.annot', 'magma/risk.genes.annot', 'magma/LDL.genes.annot', 'magma/age_smkinit.genes.annot', 'magma/maternal_smok.genes.annot']


In [6]:
#traits where job succeeded
list(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.out")))

['infant_bw',
 'age_menarche',
 'neurot',
 'age_smkinit',
 'friend_sat',
 'nsex',
 'bmi',
 'hr',
 'risk',
 'auto_speed',
 'dpw',
 'maternal_smok',
 'adhd',
 'antisoc',
 'cud',
 'height',
 'LDL',
 'addict-rf',
 'townsend',
 'alc_dep',
 'facial_hair']

In [7]:
#traits where job failed
rerun=set(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.annot") ))-set(list(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.out") )))
print(rerun)

set()


all initally failed jobs don't have rsids- required formatting apparently, for use of 1000 genomes. 
none have CHR X and Y- all are numerically ordered

only antisoc failed- there is a pvalue that isn't parsable

## merge data dictionary for traits without SNPs with SNP dictionary to add RSIDs

In [137]:
snp=pd.read_csv('~/bsl/magma_v1/g1000_eur/g1000_eur.bim',sep='\t',header=None)
snp.columns=['CHR','RSID','CM','BP','A1','A2']

In [138]:
snp['marker_merge']=(snp['CHR']).map(lambda x:str(x))+':'+(snp['BP']).map(lambda x:str(x))+':'+snp['A1']+':'+snp['A2']

In [139]:
snp

,CHR,RSID,CM,BP,A1,A2,marker_merge
0,1,rs537182016,0,10539,A,C,1:10539:A:C
1,1,rs575272151,0,11008,G,C,1:11008:G:C
2,1,rs544419019,0,11012,G,C,1:11012:G:C
3,1,rs540538026,0,13110,A,G,1:13110:A:G
4,1,rs62635286,0,13116,G,T,1:13116:G:T
...,...,...,...,...,...,...,...
22665059,23,rs148015245,0,154928909,T,C,23:154928909:T:C
22665060,23,rs140888996,0,154929192,T,G,23:154929192:T:G
22665061,23,rs557132,0,154929412,T,C,23:154929412:T:C
22665062,23,rs76850183,0,154929639,C,T,23:154929639:C:T


## fix antisocial error

In [ ]:
t='antisoc'

In [ ]:
file_dict[t]['marker_merge']=file_dict[t][colname_dict[t][1]].map(lambda x:str(x))+':'+file_dict[t][colname_dict[t][2]].map(lambda x:str(x))+':'+file_dict[t][colname_dict[t][4]].str.upper()+':'+file_dict[t][colname_dict[t][5]].str.upper()
file_dict[t]=file_dict[t].merge(snp[['RSID','marker_merge']],on='marker_merge',how='inner')

,MarkerName,Allele1,Allele2,Weight,Zscore,P-value,Direction,CHR,BP,marker_merge,RSID
1283482,14:22262789,t,g,16400.0,0.9190,0.3583,++--+,14,22262789,14:22262789:T:G,rs12886640
1283483,6:50483161,a,c,16400.0,-0.2930,0.7693,--+-+,6,50483161,6:50483161:A:C,rs79400560
1283484,2:134909570,t,c,1379.0,0.6938,--+++,NaN,2,134909570,2:134909570:T:C,rs113396222
1283485,11:126026645,t,c,16400.0,-0.4780,0.6326,-+-+-,11,126026645,11:126026645:T:C,rs12421177


In [62]:
#remove line with missing pvalue- and remove any others that may have non-numeric p-values
file_dict[t]['P-value']=pd.to_numeric(file_dict[t]['P-value'],'coerce')
file_dict[t]=file_dict[t].dropna(subset=['P-value'])

In [64]:
file_dict[t].loc[1283482:1283485] # check line 1283485- one that threw the error- it's missing a pvalue

,MarkerName,Allele1,Allele2,Weight,Zscore,P-value,Direction,CHR,BP,marker_merge,RSID
1283482,14:22262789,t,g,16400.0,0.919,0.3583,++--+,14,22262789,14:22262789:T:G,rs12886640
1283483,6:50483161,a,c,16400.0,-0.293,0.7693,--+-+,6,50483161,6:50483161:A:C,rs79400560
1283485,11:126026645,t,c,16400.0,-0.478,0.6326,-+-+-,11,126026645,11:126026645:T:C,rs12421177


In [65]:
pos,pval=ex_magma(file_dict[t],colname_dict[t],int(meta_data[meta_data['label']==t]['N']),t,'magma/',True)

           SNP CHR        POS
0     rs667647   5   29439275
1  rs113534962   5   85928892
2    rs2366866  10  128341232
3     rs472303   3   62707519
4   rs11725240   4   55643311
pos file written to magma/antisoc_pos.tsv
16400
       N          SNP       P
0  16400     rs667647  0.1959
1  16400  rs113534962  0.9342
2  16400    rs2366866  0.3463
3  16400     rs472303  0.6594
4  16400   rs11725240  0.8004
pval file written to magma/antisoc_pval.tsv


# merge in gene names

In [ ]:
gene_loc=pd.read_csv('/tscc/projects/ps-palmer/brittany/magma_v1/NCBI38/NCBI38.gene.loc',sep='\t',header=None)
gene_loc.columns=['ID','CHR','START','STOP','STAND','GENE_NAME']

In [ ]:
file_ls=file_dict.keys()

In [15]:
file_ls=['cud', 'adhd', 'alc_dep', 'dpw', 'risk', 'auto_speed', 'nsex', 'bmi', 'height']

In [17]:
magma_dict={}
for t in file_ls:
    magma_dict[t]=pd.read_csv('magma/'+t+'.genes.out',delim_whitespace=True)
    magma_dict[t]=gene_loc[['ID','GENE_NAME']].merge(magma_dict[t], left_on='ID',right_on='GENE',how='right').reset_index().drop(['index','GENE'],axis=1)
    magma_dict[t].columns=['ID', 'GENE', 'CHR', 'START', 'STOP', 'NSNPS', 'NPARAM', 'N','ZSTAT', 'P']
    magma_dict[t]['Q']=statsmodels.stats.multitest.fdrcorrection(magma_dict[t]['P'],alpha=0.05,method='indep',is_sorted=False)[1]
    magma_dict[t].to_csv('magma/seed_genes/'+t+'_annot.tsv',sep='\t',index=False)